In [2]:
import uproot
import numpy as np
import matplotlib.pylab as plt
import matplotlib.colors as colors
from scipy.optimize import curve_fit
# import file_reading from src file 
from src.file_reading import read_file
from importlib import reload


In [4]:
path = 'data/' # set this to '' to run on the GitHub version
events_sim = uproot.open(path+'PhaseSpaceSimulation.root')
events_down = uproot.open(path+'B2HHH_MagnetDown.root')
events_up = uproot.open(path+'B2HHH_MagnetUp.root')

In [28]:
list_of_interesting_keys = []
for i in range(1, 3):
    exec(f"list_of_interesting_keys.append('H{i}_PX')")
    exec(f"list_of_interesting_keys.append('H{i}_PY')  ")
    exec(f"list_of_interesting_keys.append('H{i}_PZ')  ")
    exec(f"list_of_interesting_keys.append('H{i}_Charge')  ")
    exec(f"list_of_interesting_keys.append('H{i}_ProbK')  ")
    exec(f"list_of_interesting_keys.append('H{i}_ProbPi')  ")
    exec(f"list_of_interesting_keys.append('H{i}_isMuon')  ")

In [33]:
# Check what's in the tree. 
# Note that the simulation tree is called 'PhaseSpaceTree' and does not have the ProbPi/K variables filled.
print('Input data variables:')
print(events_up['DecayTree'].keys())

# These are the arrays to hold the data
pT = []
pX = []
pY = []
pZ = []

# A counter for bookkeeping
event_counter = 0

# If set to a value greater than 0, limits the number of events analysed
# Set to -1 to run over all events. 
# It is recommended to keep the number of events limited while developing the analysis.
MAX_EVENTS = 50000

# Select which set of input data is to be analysed. Uncomment exactly one line
trees = [events_sim['PhaseSpaceTree']]                       # Simulation
#trees = [events_down[b'DecayTree']]                          # Magnet down data
#trees = [events_up['DecayTree']]                             # Magnet up data
#trees = [events_down[b'DecayTree'],events_up['DecayTree']]   # Magnet down+up data

# This loop goes over the trees to be analysed
for tree in trees:
    # This outer loop is a technical loop of uproot over chunks of events
    
    for data in tree.iterate(list_of_interesting_keys):
        # As Python can handle calculations with arrays, we can calculate derived quantities here
        print(type(data[b'H1_PX']))
        print(data[b'H1_PX'] ** 2)
        pT_H1 = np.sqrt(data[b'H1_PX']**2+data[b'H1_PY']**2)
        pT_H2 = np.sqrt(data[b'H2_PX']**2+data[b'H2_PY']**2)
        pT_H3 = np.sqrt(data[b'H3_PX']**2+data[b'H3_PY']**2)

        # Your invariant mass calculation should go here
        
        # This loop will go over individual events
        for i in range(0,len(data[b'H1_PZ'])):
            event_counter += 1
            if 0 < MAX_EVENTS and MAX_EVENTS < event_counter: break
            if 0 == (event_counter % 100000): print('Read', event_counter, 'events')
            # Decide here which events to analyse
            if (data[b'H1_PZ'][i] < 0) or (data[b'H2_PZ'][i] < 0) or (data[b'H3_PZ'][i] < 0): continue
            # Fill arrays of events to be plotted and analysed further below
            # Adding values for all three hadrons to the same variable here
            pT.append(pT_H1[i])
            pT.append(pT_H2[i])
            pT.append(pT_H3[i])
            pX.append(data[b'H1_PX'][i])
            pX.append(data[b'H2_PX'][i])
            pX.append(data[b'H3_PX'][i])
            pY.append(data[b'H1_PY'][i])
            pY.append(data[b'H2_PY'][i])
            pY.append(data[b'H3_PY'][i])
            pZ.append(data[b'H1_PZ'][i])
            pZ.append(data[b'H2_PZ'][i])
            pZ.append(data[b'H3_PZ'][i])
 
print('Read {:d} events'.format(event_counter))

Input data variables:
['B_FlightDistance', 'B_VertexChi2', 'H1_PX', 'H1_PY', 'H1_PZ', 'H1_ProbK', 'H1_ProbPi', 'H1_Charge', 'H1_IPChi2', 'H1_isMuon', 'H2_PX', 'H2_PY', 'H2_PZ', 'H2_ProbK', 'H2_ProbPi', 'H2_Charge', 'H2_IPChi2', 'H2_isMuon', 'H3_PX', 'H3_PY', 'H3_PZ', 'H3_ProbK', 'H3_ProbPi', 'H3_Charge', 'H3_IPChi2', 'H3_isMuon']
<class 'awkward.highlevel.Array'>
[1.26e+07, 6.38e+06, 4.91e+05, 1.13e+07, ..., 1.54e+06, 3.54e+08, 2.58e+09]


ValueError: while calling

    numpy.power.__call__(
        <Array [{H1_PX: 5.21e+03, ...}, ..., {...}] type='5 * {H1_PX: float...'>
        2
    )

Error details: cannot broadcast records in power

In [13]:
#reload file_reading
#pT, pX, pY, pZ = read_file(MAX_EVENTS=5000, mode=0)

In [17]:
for tree in trees:
    keys = tree.keys()
    print(keys)
    desired_keys = ['H1_PX','H1_PY', 'H1_PZ','H1_Charge','H1_Prob*','H1_isMuon', 'H2_PX', "H2_PY", "H2_PZ", 'H2_Charge','H2_Prob*','H2_isMuon', 'H3_PX','H3_PY','H3_PZ','H3_Charge','H3_Prob*','H3_isMuon']
    for data in tree.iterate(desired_keys):
        print(tree)

['B_FlightDistance', 'B_VertexChi2', 'H1_PX', 'H1_PY', 'H1_PZ', 'H1_ProbK', 'H1_ProbPi', 'H1_Charge', 'H1_IPChi2', 'H1_isMuon', 'H2_PX', 'H2_PY', 'H2_PZ', 'H2_ProbK', 'H2_ProbPi', 'H2_Charge', 'H2_IPChi2', 'H2_isMuon', 'H3_PX', 'H3_PY', 'H3_PZ', 'H3_ProbK', 'H3_ProbPi', 'H3_Charge', 'H3_IPChi2', 'H3_isMuon']


SyntaxError: invalid syntax
in file data/PhaseSpaceSimulation.root
in object /PhaseSpaceTree;1 (<unknown>, line 1)

In [22]:
list_of_interesting_keys = []
for i in range(3):
    exec(f"list_of_interesting_keys.append('H{i}_PX')")
    exec(f"list_of_interesting_keys.append('H{i}_PY')  ")
    exec(f"list_of_interesting_keys.append('H{i}_PZ')  ")
    exec(f"list_of_interesting_keys.append('H{i}_Charge')  ")
    exec(f"list_of_interesting_keys.append('H{i}_Prob{i}')  ")
    exec(f"list_of_interesting_keys.append('H{i}_isMuon')  ")

In [23]:
list_of_interesting_keys

['H0_PX',
 'H0_PY',
 'H0_PZ',
 'H0_Charge',
 'H0_Prob0',
 'H0_isMuon',
 'H1_PX',
 'H1_PY',
 'H1_PZ',
 'H1_Charge',
 'H1_Prob1',
 'H1_isMuon',
 'H2_PX',
 'H2_PY',
 'H2_PZ',
 'H2_Charge',
 'H2_Prob2',
 'H2_isMuon']

In [1]:
        print(type(data[b'H1_PX']))
data[b'H1_PX']

NameError: name 'data' is not defined